In [ ]:
#import libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geocoder --yes
import geocoder

Get data from wiki:
take html page, cit out table with postal code data
fit it first into list and then into the dataframe.

In [ ]:
#get data from wiki as html page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

#cut out postalcode table from html page
startTable=results.text.find('<table class="wikitable sortable">\n') #where data i need starts
EndTable=results.text.find('</table>') #where data i need ends
tableData=results.text[startTable:EndTable]

#get column header names
tHeaders=tableData[tableData.find('<tr>\n<th>'):tableData.find('</th></tr>')]
tHeaders=tHeaders.replace('<tr>','').replace('</th>','').replace("\n", "")
hList=tHeaders.split('<th>')
hList.pop(0)

#get table data into list
tData=tableData[tableData.find('</th></tr>\n')+11:tableData.find('</tr></tbody>')+5]
tList=tData.replace("\n", "").split('</tr>')
i=0
#dData=pd.DataFrame
for n in tList:
    tList[i]=tList[i].replace('<tr>','')
    tList[i]=tList[i].replace('</td>','')
    tList[i]=tList[i].split('<td>')
    tList[i].pop(0)
    i=i+1
tList = list(filter(None, tList))

# create dataframe
dData=pd.DataFrame(tList)
dData.columns=hList

clean data :
remove not assigned borough rows as we dont analyse them
check not assigned neighborough to assign them borough name

In [ ]:
#remove not assigned postal code to Borough
dData = dData[dData.Borough != 'Not assigned']

if (dData[dData.Neighbourhood == 'Not assigned'].shape[0]==0): 
    print('no need to assign borough names to neighborough ')
dData.set_index('Postal Code',inplace=True)
dData=dData.reset_index()
#dData.tail()

In [47]:
#get size of dataframe
print ('size of dataframe is:', dData.shape)

size of dataframe is: (103, 3)


In [ ]:
#get csv file because I couldnot get geodata from geocoder

!wget -q -O 'toronto_data.csv' 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
print('Data downloaded!')

In [ ]:
geocsv_df = pd.read_csv('toronto_data.csv')
geocsv_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
#geocsv_df.head(2)

In [60]:
i=dData.shape[0]
while i>0:
    i=i-1
    s=dData.loc[i,'Postal Code']    
    idx=geocsv_df.index[geocsv_df['PostalCode']==s].tolist()
    dData.loc[i,'Latitude']=geocsv_df.loc[idx[0],'Latitude']
    dData.loc[i,'Longitude']=geocsv_df.loc[idx[0],'Longitude']
dData.head(12)

,Postal Code,Borough,Neighbourhood,latitude,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,43.657162,-79.378937
